In [1]:
import os
os.getcwd()

'/teamspace/studios/this_studio/text-summarizer/research'

In [2]:
os.chdir("../")
os.getcwd()

'/teamspace/studios/this_studio/text-summarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTrasformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)
        
        
        os.makedirs(self.config.artifact_root, exist_ok=True)
        
    def get_data_transformation_config(self)-> DataTrasformationConfig:
        config = self.config.data_transformation
        
        os.makedirs(self.config.artifact_root, exist_ok=True)
        
        return DataTrasformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )
        
    

In [8]:
import os
from src.textSummarizer.logging import create_logger
from transformers import AutoTokenizer
from datasets import load_from_disk

## Data Transformation Component

In [18]:
class DataTransformation:
    def __init__(self, config: DataTrasformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
        self.logger = create_logger(self.__class__.__name__)
        
    def conver_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], truncation=True, padding='max_length', max_length=1024)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], truncation=True, padding='max_length', max_length=128)

        encodings = {
            'input_ids': input_encodings.input_ids,
            'attention_mask': input_encodings.attention_mask,
            'labels': target_encodings.input_ids
        }
        return encodings

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.conver_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, 'samsum_dataset'))

In [19]:
config = ConfigurationManager()

data_transformation_config = config.get_data_transformation_config()

data_transformation = DataTransformation(data_transformation_config)

data_transformation.convert()

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]